## MOUNTS N STUFF

In [2]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/My Drive/MSc_project/.MAIN

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/MSc_project/.MAIN


In [ ]:
# !pip install mslib
!pip install datasets
!pip install language_tool_python
!pip install pip install tweet-preprocessor
!pip install pycountry
!pip install pandas==1.2.4
!pip install tensorflow
!pip install transformers
!pip install xgboost
!pip install torch
!pip install tensorflow

In [4]:
from enum import unique
# from msilib import sequence
from textblob import TextBlob
import sys
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import os.path
import nltk
import nltk.data
import time
import string
from datasets import load_dataset

import json
import pickle

import language_tool_python
import preprocessor as p

import pycountry
import re
import string

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import random
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import os
# import readability

## DATA
from datasets import Dataset



# user genders 
import torch
from torch import nn
from nltk.stem import PorterStemmer # for stemming
from nltk.stem import WordNetLemmatizer # for lemmatization
from nltk.corpus import stopwords
from transformers import BertModel, AutoModel, AdamW, get_linear_schedule_with_warmup, BertTokenizer, BertForSequenceClassification
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

nltk.download('omw-1.4')

import tensorflow as tf
import torch

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [5]:
import tensorflow as tf
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [6]:
df = pd.read_csv('user_gender_class/test_data.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (0,1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## RUN

In [7]:
df.columns

Index(['Unnamed: 0', 'user_id', 'description', 'text', 'clean_description',
       'clean_text', 'sep', 'txt'],
      dtype='object')

In [43]:
from user_gender_class import gender_classifier as gc

df_t = df.iloc[0:100]

model_name = 'bert-base-uncased'
SEQ_LEN = 512
batch_size = 16
epochs = 2
learning_rate = 1e-5 # Controls how large a step is taken when updating model weights during training.
steps_per_epoch = int(len(df)/16)
num_workers = 3

model = gc.BERT_Arch(2)

path_model = 'user_gender_class/gender_model'
model.load_state_dict(torch.load(path_model))
# model.to(device)

tokenizer = BertTokenizer.from_pretrained(model_name, do_lower_case=True)

tokens = tokenizer.batch_encode_plus(
  df_t.txt.to_list(),
  max_length=SEQ_LEN,
  add_special_tokens=True, # Add '[CLS]' and '[SEP]'
  return_token_type_ids=True,
  truncation=True,
  padding='longest',
  return_attention_mask=True,
)

test_seq = torch.tensor(tokens['input_ids'])
test_mask = torch.tensor(tokens['attention_mask'])
test_ids = torch.tensor(tokens['token_type_ids'])

output = model(input_ids = test_seq,
                    attention_mask = test_mask, 
                    token_type_ids= test_ids)

preds = output.detach().cpu().numpy()
predictions = np.argmax(preds,axis=1)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [46]:
print(np.array(predictions.sum()))

55
